# 1. Import dependencies

In [1]:
import copy
import cv2 as cv
import csv
import mediapipe as mp
import numpy as np
import itertools
import os

from collections import deque

# 2. Initialization

## a. Initialize variables and models

In [2]:
# Initialize Mediapipe's hand model 
mediapipe_hands = mp.solutions.hands
mediapipe_drawing = mp.solutions.drawing_utils
mediapipe_drawing_styles = mp.solutions.drawing_styles
hands = mediapipe_hands.Hands(
    static_image_mode=True,   
    max_num_hands=1,
    min_detection_confidence=0.5,
    model_complexity=1,
)

# Initialize main directory 
path = os.getcwd()
main_directory = os.path.dirname(path)

## b. Initialize functions

In [3]:
# Extract & convert default-normalized landmark keys into absolute pixel value
def calc_landmark_list(image, hand_landmarks):

    # Initialize image size & new landmark list
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_list = []

    # Extract & for each landmark keys from detected hand
    for _, landmark in enumerate(hand_landmarks.landmark):

        # Convert pre-normalized landmark keys into original pixel value
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        # ! landmark_z is unused due to a bug from Mediapipe Hands when detecting the depth of the hand
        # ! and detecting 2 - dimensional datas are much easier to read and explain.
        # landmark_z = landmark.z

        # Put the converted landmark keys inside the new landmark list
        landmark_list.append([landmark_x, landmark_y])

    return landmark_list

In [77]:
# Convert into wrist-relative point coordinates & normalize keys
def pre_process_landmark(landmark_list):

    # Receive landmark list from calc_landmark_list function
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Initialize reference key
    base_x, base_y = 0, 0

    # For each detected landmark keys in landmark list
    for index, landmark_point in enumerate(temp_landmark_list):
        # If the first index of the landmark list (wrist) is detected,
        # set the corresponding landmark keys to reference key
        if index == 0:
            #base_x, base_y = landmark_point[0], landmark_point[1]
            base_x, base_y = landmark_point[0], landmark_point[0]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        #temp_landmark_list[index][1] = base_y - temp_landmark_list[index][1]
        temp_landmark_list[index][1] = base_y - temp_landmark_list[index][1]

    # Convert to a one-dimensional matrix list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Find the max value inside the one-dimensional landmark list
    max_value = max(list(map(abs, temp_landmark_list)))

    # Normalize the relative keys based from the max value
    def normalize_value(n):
        return n / max_value

    # Place & replace landmark list key with new normalized value
    temp_landmark_list = list(map(normalize_value, temp_landmark_list))

    # Output : return with the new temp_landmark_list value
    return temp_landmark_list

## c. Initialize dataset template function

In [78]:
# Initialize number of hand keypoints
num_coords = 21

# Label the corresponding sets of keypoints
alphabet = "B"

In [79]:
# Initialize .csv file
def init_csv(feature):
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]

    with open(main_directory +'/dataset/03_csv_pre_combined/keypoints_test_{}.csv'.format(feature), mode='w', newline='') as f:
    # with open(main_directory +'/dataset/csv_pre_combined/keypoints_test.csv'.format(feature), mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

## d. Initialize dataset logging function

In [80]:
def logging_csv(feature, landmark_list):

    csv_path = main_directory + '/dataset/03_csv_pre_combined/keypoints_test_{}.csv'.format(feature)
    # csv_path = main_directory + '/dataset/keypoints_test.csv'
    
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([feature, *landmark_list])

    return

## e. Initialize image location

In [81]:
IMAGE_FILES = ['SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (1).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (2).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (3).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (4).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (5).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (6).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (7).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (8).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (9).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (10).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (11).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (12).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (13).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (14).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (15).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (16).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (17).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (18).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (19).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (20).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (21).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (22).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (23).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (24).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (25).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (26).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (27).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (28).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (29).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (30).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (31).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (32).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (33).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (34).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (35).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (36).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (37).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (38).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (39).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (40).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (41).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (42).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (43).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (44).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (45).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (46).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (47).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (48).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (49).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (50).jpg'.format(alphabet, alphabet)]

# 3. Capture, re-normalize & write landmark keypoints into dataset

In [82]:
counter = 1

for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv.flip(cv.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    
    if not results.multi_hand_landmarks:
        print("[[!]] Not detected : Image " + alphabet, counter)
        counter += 1
        continue
    
    image_height, image_width, _ = image.shape
    debug_image = copy.deepcopy(image)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:

            # Convert pre-normalized landmark keys into pixels numbering
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)
            
            # Convert into relative coordinates / normalize keys from wrist point
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            
            # Write/log received keypoints into dataset
            logging_csv(alphabet, pre_processed_landmark_list)
            print('Detected : Image '+ alphabet, counter)
            counter += 1
            # Visualize complete hand landmarks
            mediapipe_drawing.draw_landmarks(
                debug_image,
                hand_landmarks,
                mediapipe_hands.HAND_CONNECTIONS,
                mediapipe_drawing_styles.get_default_hand_landmarks_style(),
                mediapipe_drawing_styles.get_default_hand_connections_style())

Detected : Image B 1
Detected : Image B 2
Detected : Image B 3
Detected : Image B 4
Detected : Image B 5
Detected : Image B 6
Detected : Image B 7
Detected : Image B 8
Detected : Image B 9
Detected : Image B 10
Detected : Image B 11
Detected : Image B 12
Detected : Image B 13
Detected : Image B 14
Detected : Image B 15
Detected : Image B 16
Detected : Image B 17
Detected : Image B 18
Detected : Image B 19
Detected : Image B 20
Detected : Image B 21
Detected : Image B 22
Detected : Image B 23
Detected : Image B 24
Detected : Image B 25
Detected : Image B 26
Detected : Image B 27
Detected : Image B 28
Detected : Image B 29
Detected : Image B 30
Detected : Image B 31
Detected : Image B 32
Detected : Image B 33
Detected : Image B 34
Detected : Image B 35
[[!]] Not detected : Image B 36
[[!]] Not detected : Image B 37
Detected : Image B 38
[[!]] Not detected : Image B 39
Detected : Image B 40
Detected : Image B 41
Detected : Image B 42
Detected : Image B 43
Detected : Image B 44
Detected : 

In [76]:
print(landmark_list[0])

[1756, 1687]
